In [2]:
import qutip as qt
import numpy as np
print(qt.__version__)



5.0.4


## First we define state and density matrix

In [3]:

def create_w_state(n):
    basis_states = [qt.basis([2]*n, [1 if i == j else 0 for i in range(n)]) for j in range(n)]
    w_state = sum(basis_states) / np.sqrt(n)
    return w_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3 
w_state = create_w_state(num_qubits)
p_W = compute_density_matrix(w_state)
S_W = qt.entropy_vn(p_W)

print("W state:")
print(w_state)
np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:\n")
print(np.real(p_W.full()))
print("\n Entropy of the total system S(A,B,C) = ",S_W)


W state:
Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[0.        ]
 [0.57735027]
 [0.57735027]
 [0.        ]
 [0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]]
Density Matrix in Matrix Form:

[[0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.333 0.333 0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.   ]]

 Entropy of the total system S(A,B,C) =  -4.269758091753807e-15


## Then we compute partial traces to get rdms and compute entropies

In [4]:

p_A = p_W.ptrace([0])  
p_B = p_W.ptrace([1])  
p_C = p_W.ptrace([2])  


S_A = qt.entropy_vn(p_A)
S_B = qt.entropy_vn(p_B)
S_C = qt.entropy_vn(p_C)

I_ABC = S_A + S_B + S_C - S_W

np.set_printoptions(linewidth=200, precision=3, suppress=True)

print("Reduced Density Matrix ρ_A:\n", np.real(p_A.full()), "\n",f"Von Neumann Entropy S(ρ_A): {S_A:.6f} \n")
print("Reduced Density Matrix ρ_B:\n", np.real(p_B.full()), "\n",f"Von Neumann Entropy S(ρ_B): {S_B:.6f} \n")
print("Reduced Density Matrix ρ_C:\n", np.real(p_C.full()), "\n",f"Von Neumann Entropy S(ρ_C): {S_C:.6f} \n")
print("\n \n Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) = ",I_ABC)

Reduced Density Matrix ρ_A:
 [[0.667 0.   ]
 [0.    0.333]] 
 Von Neumann Entropy S(ρ_A): 0.636514 

Reduced Density Matrix ρ_B:
 [[0.667 0.   ]
 [0.    0.333]] 
 Von Neumann Entropy S(ρ_B): 0.636514 

Reduced Density Matrix ρ_C:
 [[0.667 0.   ]
 [0.    0.333]] 
 Von Neumann Entropy S(ρ_C): 0.636514 


 
 Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) =  1.9095425048844425


## Now we work with some operators correlations, first Z:

In [5]:
Z = qt.sigmaz()

Z_full = qt.tensor(Z, Z, Z)
avg_Z_full = (p_W * Z_full).tr()

avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()

Z_norm = Z.norm()

print(f"Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): {avg_Z_full:.6f}")
print(f"Tr_A(p_A * σ_z): {avg_Z_A:.6f}")
print(f"Tr_B(p_B * σ_z): {avg_Z_B:.6f}")
print(f"Tr_C(p_C * σ_z): {avg_Z_C:.6f}")
print(f"Operator norm of σ_z: {Z_norm:.6f}")


Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): -1.000000
Tr_A(p_A * σ_z): 0.333333
Tr_B(p_B * σ_z): 0.333333
Tr_C(p_C * σ_z): 0.333333
Operator norm of σ_z: 2.000000


In [6]:
C_ABC_z = avg_Z_full - avg_Z_A*avg_Z_B*avg_Z_C
f_ABC_z = (C_ABC_z**2)/(2*3*Z_norm**2)

print("Concluding for Z we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", f_ABC_z)

Concluding for Z we have that I(A,B,C) =  1.9095425048844425 While the lower bound is f_ABC =  0.04481024234110655


## Then X

In [7]:
x = qt.sigmax()

x_full = qt.tensor(x, x, x)
avg_x_full = (p_W * x_full).tr()

avg_x_A = (p_A * x).tr()
avg_x_B = (p_B * x).tr()
avg_x_C = (p_C * x).tr()

x_norm = x.norm()

print(f"Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): {avg_x_full:.6f}")
print(f"Tr_A(p_A * σ_x): {avg_x_A:.6f}")
print(f"Tr_B(p_B * σ_x): {avg_x_B:.6f}")
print(f"Tr_C(p_C * σ_x): {avg_x_C:.6f}")
print(f"Operator norm of σ_x: {x_norm:.6f}")

Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): 0.000000+0.000000j
Tr_A(p_A * σ_x): 0.000000+0.000000j
Tr_B(p_B * σ_x): 0.000000+0.000000j
Tr_C(p_C * σ_x): 0.000000+0.000000j
Operator norm of σ_x: 2.000000


In [8]:
C_ABC_x = avg_x_full - avg_x_A*avg_x_B*avg_x_C
f_ABC_x = (C_ABC_x**2)/(2*3*x_norm**2)

print("Concluding for X we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", f_ABC_x)

Concluding for X we have that I(A,B,C) =  1.9095425048844425 While the lower bound is f_ABC =  0j


## And for Y:

In [9]:
y = qt.sigmay()

y_full = qt.tensor(y, y, y)
avg_y_full = (p_W * y_full).tr()

avg_y_A = (p_A * y).tr()
avg_y_B = (p_B * y).tr()
avg_y_C = (p_C * y).tr()

y_norm = y.norm()

print(f"Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): {avg_y_full:.6f}")
print(f"Tr_A(p_A * σ_y): {avg_y_A:.6f}")
print(f"Tr_B(p_B * σ_y): {avg_y_B:.6f}")
print(f"Tr_C(p_C * σ_y): {avg_y_C:.6f}")
print(f"Operator norm of σ_y: {y_norm:.6f}")

Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): 0.000000+0.000000j
Tr_A(p_A * σ_y): 0.000000+0.000000j
Tr_B(p_B * σ_y): 0.000000+0.000000j
Tr_C(p_C * σ_y): 0.000000+0.000000j
Operator norm of σ_y: 2.000000


In [10]:
C_ABC_y = avg_y_full - avg_y_A*avg_y_B*avg_y_C
f_ABC_y = (C_ABC_y**2)/(2*3*y_norm**2)

print("Concluding for Y we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", f_ABC_y)

Concluding for Y we have that I(A,B,C) =  1.9095425048844425 While the lower bound is f_ABC =  0j


## What if I try to mix them?

In [11]:
Mix_operator = qt.tensor(x, y, Z)
full_mix_avg = (p_W * Mix_operator).tr()
C_ABC_MIX = full_mix_avg - avg_x_A*avg_y_B*avg_Z_C
f_ABC_MIX = (C_ABC_MIX**2)/(2*x_norm**2*y_norm**2*Z_norm**2)
print("Concluding for a mix of the 3 operators we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", f_ABC_MIX)

Concluding for a mix of the 3 operators we have that I(A,B,C) =  1.9095425048844425 While the lower bound is f_ABC =  0j
